In [7]:
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
load_dotenv()

True

In [6]:
zip_code = os.environ.get('ZIP')

driver = webdriver.Firefox()
driver.get("https://www.wholefoodsmarket.com/products/all-products?featured=on-sale")
assert "Whole Foods" in driver.title
try :
    store = driver.find_element(by=By.ID, value="pie-store-finder-modal-search-field")
    store.clear()
    store.send_keys(zip_code)
    store.send_keys(Keys.RETURN)
    closest = driver.find_element(by=By.CSS_SELECTOR, value='li.wfm-search-bar--list_item:nth-child(1)')
    closest.send_keys(Keys.RETURN)
except:
    
driver.close()

C:\Users\Corey\AppData\Local\Temp\ipykernel_14128\516899527.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  store = driver.find_element_by_id("pie-store-finder-modal-search-field")
C:\Users\Corey\AppData\Local\Temp\ipykernel_14128\516899527.py:10: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  closest = driver.find_element_by_css_selector('li.wfm-search-bar--list_item:nth-child(1)')


NoSuchElementException: Message: Unable to locate element: li.wfm-search-bar--list_item:nth-child(1)
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:395:5
element.find/</<@chrome://remote/content/marionette/element.js:300:16
